In [1]:
import $file.rec4stdlib
import rec4stdlib._

Compiling /Users/Eli/Dropbox/FALL 2019 COURSES/CSCI 3155 Principles of Programming Languages/Recitation/recitation_week4/private_recitation_week4/rec4stdlib.sc

import $file.$         

import rec4stdlib._

# Recitation: Week 5

This week's recitation is a review. Work these problems with a group in whichever order you prefer, asking your TA for assistance when you need help.

These problems involve the following definition of binary trees:

$$
\begin{align}
BTree\ a\ :=&\ \text{Leaf} \\
          \mid&\ \text{Branch}\ (BTree\ a)\ a\ (BTree\ a)\
\end{align}
$$

In [2]:
sealed trait BTree[+A]
case object Leaf extends BTree[Nothing]
case class Branch[A](left: BTree[A], data: A, right: BTree[A]) extends BTree[A]

defined trait BTree
defined object Leaf
defined class Branch

## 1
*Inductive values*

Create the following tree as a value in Scala:
```
    5
   / \
  4   3
 /   / \
2   7   5
```

In [9]:
val root3 = Branch(Branch(Leaf, 7, Leaf), 3, Branch(Leaf, 5, Leaf)) // This is all broken apart to better see each side bottom to top
val root4 = Branch(Branch(Leaf, 2, Leaf), 4, Leaf)
val ans1 = Branch(root4, 5, root3)

root3: Branch[Int] = Branch(Branch(Leaf, 7, Leaf), 3, Branch(Leaf, 5, Leaf))
root4: Branch[Int] = Branch(Branch(Leaf, 2, Leaf), 4, Leaf)
ans1: Branch[Int] = Branch(
  Branch(Branch(Leaf, 2, Leaf), 4, Leaf),
  5,
  Branch(Branch(Leaf, 7, Leaf), 3, Branch(Leaf, 5, Leaf))
)

## 2a
*Recursion and pattern matching*

Write the following function (for use in 2b):
$$\text{max}: Nat \rightarrow Nat \rightarrow Nat$$
which returns the greater of the two inputs.

In [10]:
def max(x: Nat, y: Nat): Nat = (x, y) match {
    case (Zero, _) => y
    case (_, Zero) => x
    case (Succ(px), Succ(py)) => Succ(max(px, py))
}

defined function max

In [8]:
assert(max(one, two) == two)
assert(max(five, Zero) == five)
assert(max(five, five) == five)

## 2b
*More recursion and polymorphism*

Write the following function:
$$\text{height}: BTree \rightarrow Nat$$
which gives the height of a tree. Check the height of the tree you made in 1a to double check.

In [6]:
def height[A](t: BTree[A]): Nat = t match {
    case Leaf => Zero
    case Branch(l, _, r) => Succ(max(height(l), height(r)))
}

defined function height

In [7]:
assert(height(Leaf) == Zero)
assert(height(Branch(Leaf, five, Branch(Leaf, one, Leaf))) == two)

## 3c
*filter and lambdas*

Keep only trees from the following list with height less than or equal to 1 (hint: use a stdlib function).

In [8]:
val t1 = Leaf // 0
val t2 = Branch(t1, one, t1) // 1
val t3 = Branch(t1, two, t2) // 2
val t4 = Branch(t3, two, t3) // 3
val t5 = Branch(t2, two, t1) // 2
val l = Cons(t1, Cons(t2, Cons(t3, Cons(t4, Cons(t5, Empty)))))

t1: Leaf.type = Leaf
t2: Branch[Succ] = Branch(Leaf,Succ(Zero),Leaf)
t3: Branch[Succ] = Branch(Leaf,Succ(Succ(Zero)),Branch(Leaf,Succ(Zero),Leaf))
t4: Branch[Succ] = Branch(Branch(Leaf,Succ(Succ(Zero)),Branch(Leaf,Succ(Zero),Leaf)),Succ(Succ(Zero)),Branch(Leaf,Succ(Succ(Zero)),Branch(Leaf,Succ(Zero),Leaf)))
t5: Branch[Succ] = Branch(Branch(Leaf,Succ(Zero),Leaf),Succ(Succ(Zero)),Leaf)
l: Cons[Product with Serializable with BTree[Succ]] = Cons(Leaf,Cons(Branch(Leaf,Succ(Zero),Leaf),Cons(Branch(Leaf,Succ(Succ(Zero)),Branch(Leaf,Succ(Zero),Leaf)),Cons(Branch(Branch(Leaf,Succ(Succ(Zero)),Branch(Leaf,Succ(Zero),Leaf)),Succ(Succ(Zero)),Branch(Leaf,Succ(Succ(Zero)),Branch(Leaf,Succ(Zero),Leaf))),Cons(Branch(Branch(Leaf,Succ(Zero),Leaf),Succ(Succ(Zero)),Leaf),Empty)))))

In [9]:
val ans2c = filter((x: BTree[Nat]) => lte(height(x), one), l)

ans2c: List[Product with Serializable with BTree[Succ]] = Cons(Leaf,Cons(Branch(Leaf,Succ(Zero),Leaf),Empty))

In [10]:
assert(ans2c == Cons(Leaf,Cons(Branch(Leaf,Succ(Zero),Leaf),Empty)))

## 3a
*map and higher order functions*


`map` is not only definable on lists, it can be used with many different data structures. For this problem, define map for BTrees:
$$\text{map_tree}: (a \rightarrow b) \rightarrow BTree\ a \rightarrow BTree\ b$$

In [11]:
def map_tree[A, B](f: A => B, t: BTree[A]): BTree[B] = t match {
    case Leaf => Leaf
    case Branch(l, d, r) => Branch(map_tree(f, l), f(d), map_tree(f, r))
}

defined function map_tree

In [12]:
assert(map_tree(Succ(_: Nat), Leaf) == Leaf)

## 3b
*map and lambdas*

Change the following tree by negating each element (maybe using a function you just defined... hint hint).

In [13]:
val t = Branch(Branch(Leaf, True, Leaf), False, Branch(Leaf, False, Branch(Leaf, True, Leaf)))
val ans3b = map_tree(not(_: Bool), t)

t: Branch[Product with Serializable with Bool] = Branch(Branch(Leaf,True,Leaf),False,Branch(Leaf,False,Branch(Leaf,True,Leaf)))
ans3b: BTree[Bool] = Branch(Branch(Leaf,False,Leaf),True,Branch(Leaf,True,Branch(Leaf,False,Leaf)))

In [14]:
assert(ans3b == Branch(Branch(Leaf,False,Leaf),True,Branch(Leaf,True,Branch(Leaf,False,Leaf))))

## 4a
*Pattern matching*

We can address nodes in many ways, for this problem, we will use a list of numbers, with a 0 meaning take a left and a 1 meaning take a right. When the numbers run out, you've reached the desired node. Define the following function that implements this addressing scheme:

$$\text{get_at_addr}: List\ Nat \rightarrow BTree\ a \rightarrow a$$

If you reach a Leaf (no more data) return `???` (this throws a `NotImplementedError`).

In [15]:
def get_at_addr[A](path: List[Nat], t: BTree[A]): A = t match {
    case Leaf => ???
    case Branch(l, d, r) => path match {
        case Empty => d
        case Cons(Zero, rest) => get_at_addr(rest, l)
        case Cons(_, rest) => get_at_addr(rest, r)
    }
}

defined function get_at_addr

In [16]:
assert(get_at_addr(Cons(one, Empty), Branch(Leaf, four, Branch(Leaf, five, Leaf))) == five)

## 4b
*Maybe*

Crashing is usually not what we want on edge cases. Change `get_at_addr` to return a `Maybe[A]` (from the hw) which allows us to represent both a found value and no answer safely. Call it `get_at_addr_maybe`.

In [17]:
def get_at_addr_maybe[A](path: List[Nat], t: BTree[A]): Maybe[A] = t match {
    case Leaf => None
    case Branch(l, d, r) => path match {
        case Empty => Just(d)
        case Cons(Zero, rest) => get_at_addr_maybe(rest, l)
        case Cons(_, rest) => get_at_addr_maybe(rest, r)
    }
}

defined function get_at_addr_maybe

In [18]:
assert(get_at_addr_maybe(Cons(one, Empty), Branch(Leaf, four, Branch(Leaf, five, Leaf))) == Just(five))
assert(get_at_addr_maybe(Cons(Zero, Empty), Branch(Leaf, four, Branch(Leaf, five, Leaf))) == None)

## 5

Write a function that composes two other functions. A.K.A:

$$compose(f, g)(X) == g(f(X))$$

It should have the following type:

$$\text{compose}: (a \rightarrow b) \rightarrow (b \rightarrow c) \rightarrow (a \rightarrow c)$$

In [19]:
def compose[A, B, C](f: A => B, g: B => C): A => C =
    a => g(f(a))

defined function compose

In [20]:
assert(compose(not, not)(True) == not(not(True)))

## 6
*currying, higher order functions, polymorphism*

Write a function that curries a 2 parameter function

$$\text{curry}: ((a, b) \rightarrow c) \rightarrow (a \rightarrow b \rightarrow c)$$

In [21]:
def curry[A, B, C](f: (A, B) => C): A => B => C =
    a => b => f(a, b)

defined function curry

In [22]:
assert(curry(plus(_: Nat, _: Nat))(one)(two) == three)